# Session Management in Strands

This document explains how Strands agents maintain conversation context, handle state management, and support persistent sessions across interactions.

Strands agents maintain state in several forms:

* Conversation History: The sequence of messages between the user and the agent
* Tool State: Information about tool executions and results
* Request State: Contextual information maintained within a single request

Understanding how state works in Strands is essential for building agents that can maintain context across multi-turn interactions and workflows.

## Conversation History

The primary form of state in a Strands agent is the conversation history, directly accessible through the agent.messages property:

In [2]:
from strands import Agent

# Create an agent
agent = Agent()

# Send a message and get a response
agent("Hows it hanging!?!")

# Access the conversation history
print(agent.messages)  # Shows all messages exchanged so far

Hey there! I'm doing well, thanks! Just here and ready to chat. How are you doing today? Anything specific you'd like to talk about or help with?[{'role': 'user', 'content': [{'text': 'Hows it hanging!?!'}]}, {'role': 'assistant', 'content': [{'text': "Hey there! I'm doing well, thanks! Just here and ready to chat. How are you doing today? Anything specific you'd like to talk about or help with?"}]}]


The agent.messages list contains all user and assistant messages, including tool calls and tool results. This is the primary way to inspect what's happening in your agent's conversation.

You can initialize an agent with existing messages to continue a conversation or pre-fill your Agent's context with information:

In [1]:
from strands import Agent

# Create an agent with initial messages
agent = Agent(messages=[
    {"role": "user", "content": [{"text": "Hello, my name is Strands!"}]},
    {"role": "assistant", "content": [{"text": "Hi there! How can I help you today?"}]}
])

# Continue the conversation
agent("What's my name?")


Your name is Strands, as you mentioned in your introduction.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Your name is Strands, as you mentioned in your introduction.'}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[1.5199804306030273], traces=[<strands.telemetry.metrics.Trace object at 0x78c29c787980>], accumulated_usage={'inputTokens': 37, 'outputTokens': 17, 'totalTokens': 54}, accumulated_metrics={'latencyMs': 1008}), state={})

Conversation history is automatically:

* Maintained between calls to the agent
* Passed to the model during each inference
* Used for tool execution context
* Managed to prevent context window overflow

## Conversation Manager

Strands uses a conversation manager to handle conversation history effectively. The default is the SlidingWindowConversationManager, which keeps recent messages and removes older ones when needed:

In [5]:
from strands import Agent
from strands.agent.conversation_manager import SlidingWindowConversationManager

# Create a conversation manager with custom window size
# By default, SlidingWindowConversationManager is used even if not specified
conversation_manager = SlidingWindowConversationManager(
    window_size=10,  # Maximum number of message pairs to keep
)

# Use the conversation manager with your agent
agent = Agent(conversation_manager=conversation_manager)

The sliding window conversation manager:

* Keeps the most recent N message pairs
* Removes the oldest messages when the window size is exceeded
* Handles context window overflow exceptions by reducing context
* Ensures conversations don't exceed model context limits

## Tool State

When an agent uses tools, the tool executions and results become part of the conversation state:

In [6]:
from strands import Agent
from strands_tools import calculator

agent = Agent(tools=[calculator])

# Tool use is recorded in the conversation history
agent("What is 123 * 456?")  # Uses calculator tool and records result

# You can examine the tool interactions in the conversation history
print(agent.messages)  # Shows tool calls and results

I can calculate that multiplication for you using the calculator tool.
Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 123 * 456           │                                                                            │
│  │ Result    │ 56088               │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The result of multiplying 123 × 456 is 56,088.[{'role': 'user', 'content': [{'text': 'What is 123 * 456?'}]}, {'role': 'assistant', 'content': [{'text': 'I can calculate that multiplication for you using the calculator tool.'}, {'toolUse': {'toolUseId': 'tooluse_hk-BQItSQVCpKaUCb04YHg', 'name': 'calculator', 'input': {'expression': '123 * 456', 'mode': 'evaluate'}}}]}, {'role': 'user', 'content': [{'toolResult': {'status': 'success', 'content': [{'text': 'Result: 56088'}], 'toolUseId': 'tooluse_hk-BQItSQVCpKaUCb04YHg'}}]}, {'role': 'assistant', 'content': [{'text': 'The result of multiplying 123 × 456 is 56,088.'}]}]


Tool state includes:

* Tool use requests from the model
* Tool execution parameters
* Tool execution results
* Any errors or exceptions that occurred

Direct tool calls can also be recorded in the conversation history:

In [7]:
from strands import Agent
from strands_tools import calculator

agent = Agent(tools=[calculator])

# Direct tool call with recording (default behavior)
agent.tool.calculator(expression="123 * 456")

# Direct tool call without recording
agent.tool.calculator(expression="765 / 987", record_direct_tool_call=False)

print(agent.messages)

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 123 * 456           │                                                                            │
│  │ Result    │ 56088               │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 765 / 987           │                                                                            │
│  │ Result    │ 0.7750759878        │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[{'role': 'user', 'content': [{'text': 'agent.tool.calculator direct tool call.\nInput parameters: {"expression": "123 * 456"}\n'}]}, {'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'tooluse_calculator_645847605', 'name': 'calculator', 'input': {'expression': '123 * 456'}}}]}, {'role': 'user', 'content': [{'toolResult': {'status': 'success', 'content': [{'text': 'Result: 56088'}], 'toolUseId': 'tooluse_calculator_645847605'}}]}, {'role': 'assistant', 'content': [{'text': 'agent.calculator was called'}]}]


In this example we can see that the first agent.tool.calculator() call is recorded in the agent's conversation history.

The second agent.tool.calculator() call is not recorded in the history because we specified the record_direct_tool_call=False argument.

## Request State

Each agent interaction maintains a request state dictionary that persists throughout the event loop cycles and is not included in the agent's context:

In [9]:
from strands import Agent

def custom_callback_handler(**kwargs):
    # Access request state
    if "request_state" in kwargs:
        state = kwargs["request_state"]
        # Use or modify state as needed
        if "counter" not in state:
            state["counter"] = 0
        state["counter"] += 1
        print(f"Callback handler event count: {state['counter']}")

agent = Agent(callback_handler=custom_callback_handler)

result = agent("Hi there!")

print(result.state)

Callback handler event count: 1
Callback handler event count: 2
Callback handler event count: 3
Callback handler event count: 4
Callback handler event count: 5
Callback handler event count: 6
Callback handler event count: 7
{'counter': 7}


The request state:

* Is initialized at the beginning of each agent call
* Persists through recursive event loop cycles
* Can be modified by tools and handlers
* Is returned in the AgentResult object

## Session Management

For applications requiring persistent sessions across separate interactions, Strands provides several approaches:

### 1. Object Persistence

The simplest approach is to maintain the Agent object across requests:

In [10]:
from strands import Agent

# Create agent once
agent = Agent()

# Use in multiple requests
def handle_request(user_message):
    return agent(user_message)

handle_request("Tell me a fun fact")
handle_request("Tell me a related fact")

Here's a fun fact: Octopuses have three hearts. Two of these hearts pump blood through the gills to pick up oxygen, while the third heart circulates the blood to the rest of the body. When an octopus swims, the third heart actually stops beating, which is one reason they prefer to crawl rather than swim - it's less tiring for them!Here's a related octopus fact: Not only do octopuses have three hearts, they also have blue blood! Unlike humans who have iron-based hemoglobin that makes our blood red, octopuses use a copper-based protein called hemocyanin to transport oxygen, which turns their blood blue when oxygenated. This copper-based system is actually more efficient at transporting oxygen in cold and low-oxygen environments like the deep ocean where many octopuses live.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Here's a related octopus fact: Not only do octopuses have three hearts, they also have blue blood! Unlike humans who have iron-based hemoglobin that makes our blood red, octopuses use a copper-based protein called hemocyanin to transport oxygen, which turns their blood blue when oxygenated. This copper-based system is actually more efficient at transporting oxygen in cold and low-oxygen environments like the deep ocean where many octopuses live."}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={}, cycle_durations=[3.4106698036193848, 2.4030375480651855], traces=[<strands.telemetry.metrics.Trace object at 0x713100c33e00>, <strands.telemetry.metrics.Trace object at 0x713110f28bf0>], accumulated_usage={'inputTokens': 115, 'outputTokens': 188, 'totalTokens': 303}, accumulated_metrics={'latencyMs': 5283}), state={})

### 2. Serialization and Restoration

For distributed systems or applications that can't maintain object references:

In [11]:
import json
import os
import uuid
from strands import Agent

# Save agent state
def save_agent_state(agent, session_id):
    os.makedirs("sessions", exist_ok=True)

    state = {
        "messages": agent.messages,
        "system_prompt": agent.system_prompt
    }
    # Store state (e.g., database, file system, cache)
    with open(f"sessions/{session_id}.json", "w") as f:
        json.dump(state, f)

# Restore agent state
def restore_agent_state(session_id):
    # Retrieve state
    with open(f"sessions/{session_id}.json", "r") as f:
        state = json.load(f)

    # Create agent with restored state
    return Agent(
        messages=state["messages"],
        system_prompt=state["system_prompt"]
    )

agent = Agent(system_prompt="Talk like a pirate")
agent_id = uuid.uuid4()

print("Initial agent:")
agent("Where are Octopus found? 🐙")
save_agent_state(agent, agent_id)

# Create a new Agent object with the previous agent's saved state
restored_agent = restore_agent_state(agent_id)
print("\n\nRestored agent:")
restored_agent("What did we just talk about?")

print("\n\n")
print(restored_agent.messages)  # Both messages and responses are in the restored agent's conversation history

Initial agent:
Arr, me hearty! Octopuses be found in all th' world's oceans, from th' shallow coastal waters to th' deep blue depths! These eight-armed scallywags prefer th' salty seas, hidin' in coral reefs, rocky lairs, and sandy bottoms.

Ye can spot 'em from th' chilly northern waters down to th' tropical paradises, though they be most plentiful and diverse in th' warm waters. Some crafty smaller species even make their homes in tide pools along th' shore!

These clever creatures be masters o' disguise, changin' colors faster than a pirate can bury treasure, and squeezin' their boneless bodies into the tiniest o' hidin' spots! Even th' giant Pacific octopus, which can grow bigger than a pirate's rowboat, can vanish like ghost ships in th' night!

Restored agent:
Arr, we were just jawin' about where octopuses be found in the briny deep! I told ye how these eight-armed sea beasties live in all the world's oceans, from shallow waters to the deep depths. They be most common in warm tro

### 3. Integrating with Web Frameworks

Strands agents can be integrated with web framework session management:

In [ ]:
from flask import Flask, request, session
from strands import Agent

app = Flask(__name__)
app.secret_key = "your-secret-key"

@app.route("/chat", methods=["POST"])
def chat():
    user_message = request.json["message"]

    # Initialize or restore agent conversation history from session
    if "messages" not in session:
        session["messages"] = []

    # Create agent with session state
    agent = Agent(messages=session["messages"])

    # Process message
    result = agent(user_message)

    # Update session with new messages
    session["messages"] = agent.messages

    # Return the agent's final message
    return {"response": result.message}

### Custom Conversation Management

For specialized requirements, you can implement your own conversation manager:

In [12]:
from strands.agent.conversation_manager import ConversationManager
from strands.types.content import Messages
from typing import Optional

class CustomConversationManager(ConversationManager):
    def apply_management(self, messages: Messages) -> None:
        """Apply management strategies to the messages list."""
        # Implement your management strategy
        pass

    def reduce_context(self, messages: Messages, e: Optional[Exception] = None) -> None:
        """Reduce context to handle overflow exceptions."""
        # Implement your reduction strategy
        pass